In [1]:
import sklearn
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Loading the dataset

In [2]:
iris = load_iris()

In [3]:
iris.data.shape, iris.target.shape, iris.target_names

((150, 4), (150,), array(['setosa', 'versicolor', 'virginica'], dtype='<U10'))

In [4]:
names = iris.target_names

#### - pre-processing

In [5]:
x_train, x_test, y_train, y_test= train_test_split(iris.data, iris.target)

In [6]:
for x in [x_train, y_train, x_test, y_test]:
    print (x.shape)

(112, 4)
(112,)
(38, 4)
(38,)


# Tringing Sklearn's LogisticRigression

In [7]:
% time
lrg = LogisticRegression()
lrg.fit(x_train, y_train)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.58 µs


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
accuracy_score(lrg.predict(x_train), y_train)

0.9553571428571429

In [9]:
accuracy_score(lrg.predict(x_test), y_test)

0.9736842105263158

# building pytorch model

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parameter as parameter
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [11]:
class LRG(nn.Module):
    def __init__(self, x_n):
        super(LRG, self).__init__()
        self.x_n = x_n
        torch.manual_seed(10)
        self.w = parameter.Parameter(torch.rand(x_n, 3, requires_grad=True))
        self.b = parameter.Parameter(torch.zeros(1, 3, requires_grad=True))
        nn.init.xavier_uniform(self.w)
        self.params = nn.ParameterList([self.w, self.b])
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
    
    def forward(self, x):        
        return F.softmax(x.mm(self.w) + self.b)
    
    def epoch(self, x, labels):
        self.optimizer.zero_grad()
        pred = self(x)
        loss = self.criterion(pred, labels)
        loss.backward()
        self.optimizer.step()
        
    def train(self, epochs, x, labels):
        x = torch.Tensor(x)
        labels = torch.LongTensor(labels)
        for i in range(epochs):
            self.epoch(x, labels)
            
    def evaluate(self, x, labels):
        x = torch.Tensor(x)
        labels = torch.LongTensor(labels)
        preds = self(x)
        return (preds.argmax(dim=1) == labels).sum().item()/len(labels)

In [12]:
% time
lg = LRG(4)
lg.train(3000, x_train, y_train)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [13]:
lg.evaluate(x_train, y_train)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


0.9642857142857143

In [14]:
lg.evaluate(x_test, y_test)

/home/tarrasse/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


0.9736842105263158